<a href="https://colab.research.google.com/github/Karthick47v2/speech-emotion-classifier/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

In [ ]:
X_mfcc =np.load('drive/MyDrive/SER/X_mfcc.npy')
X_mel = np.load('drive/MyDrive/SER/X_mel.npy')
y = np.load('drive/MyDrive/SER/y.npy')

In [ ]:
le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_mfcc, y_encoded, random_state=42, test_size=0.2)

In [ ]:
x_train.shape

(2457, 13, 200, 1)

In [ ]:
n_rows = 13
n_cols = 200
n_channels = 1

x_train = x_train.reshape(x_train.shape[0], n_rows, n_cols, n_channels)
x_test = x_test.reshape(x_test.shape[0], n_rows, n_cols, n_channels)

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(n_rows, n_cols, n_channels), activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(filters=48, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.GlobalAveragePooling2D())

model.add(keras.layers.Dense(units=8, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=keras.optimizers.Adam(learning_rate=1e-2))

n_epochs = 50
batch_size = 128

# checkPoint

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_data=(x_test, y_test), verbose=1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 11, 198, 32)       320       
                                                                 
 batch_normalization_18 (Bat  (None, 11, 198, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 5, 99, 32)        0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 3, 97, 48)         13872     
                                                                 
 batch_normalization_19 (Bat  (None, 3, 97, 48)        192       
 chNormalization)                                                
                                                      

In [ ]:
# 32 - loss: 1.6352 - accuracy: 0.3933 - val_loss: 1.7180 - val_accuracy: 0.3474
# 32 16 - loss: 1.0733 - accuracy: 0.6219 - val_loss: 1.3151 - val_accuracy: 0.5260
# 32 32 - loss: 0.7778 - accuracy: 0.7565 - val_loss: 1.3213 - val_accuracy: 0.5812
# 32 POOL 32 - loss: 0.2664 - accuracy: 0.9450 - val_loss: 0.6418 - val_accuracy: 0.7468
# 32 POOL 48 - loss: 0.1997 - accuracy: 0.9642 - val_loss: 0.5297 - val_accuracy: 0.7955
# 32 POOL 48 16 - loss: 0.0394 - accuracy: 0.9942 - val_loss: 0.2121 - val_accuracy: 0.9318

# LR - 1e-3 32 POOL 48 32 - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0830 - val_accuracy: 0.9805
# LR - 1e-2 32 POOL 48 32 - loss: 0.0148 - accuracy: 0.9971 - val_loss: 0.1945 - val_accuracy: 0.9416
# LR - 1e-4 32 POOL 48 32 - loss: 0.7221 - accuracy: 0.8133 - val_loss: 0.8885 - val_accuracy: 0.7370

# BATCH - 64 LR - 1e-2 32 POOL 48 32 - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1003 - val_accuracy: 0.9773